### Vighnesh Mane (210021607) - PG

In [ ]:
# (reference : Lab 08)
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [ ]:
# Google Colab Setup (reference: Lab 07)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# recall the path in Google Drive (reference: Lab 07)
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CW_Folder_PG' 
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['README.txt', 'labels', 'CW_Dataset.zip', '.DS_Store', 'Video', 'Code', 'Models', 'test_functions.ipynb']


In [ ]:
GOOGLE_DRIVE_PATH

'drive/My Drive/CW_Folder_PG'

In [ ]:
# update opencv (reference: Lab 07)
!pip install opencv-python==4.5.5.64

     |████████████████████████████████| 60.5 MB 138.8 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
# check correct opencv version (reference: Lab 07)
!pip show opencv-python

Name: opencv-python
Version: 4.5.5.64
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/skvark/opencv-python
Author: None
Author-email: None
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy
Required-by: imgaug, dopamine-rl, albumentations


In [ ]:
# copy and unzip dataset directly on the Colab server (reference: Lab 07)
# Identify path to zipped dataset
zip_path = os.path.join(GOOGLE_DRIVE_PATH, 'CW_Dataset.zip')

# Copy it to Colab
!cp '{zip_path}' .

# Unzip it
!yes|unzip -q CW_Dataset.zip

# Delete zipped version from Colab (not from Drive)
!rm CW_Dataset.zip

In [ ]:
# import libraries
import cv2
from sklearn.model_selection import train_test_split
from skimage import img_as_ubyte, io, color
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm, metrics
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision
import torch.nn.functional as F
from PIL import Image
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
%matplotlib inline

In [ ]:
from torch.utils import data
# (reference : https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#loading-a-dataset)
# creating the custom dataset for the image files
class CNN_func(Dataset):

  def __init__(self, data_file, root, transform = None):
    self.data_file = data_file
    self.root = root
    self.transform = transform

  def __len__(self):
    return len(self.data_file)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    img_path = os.path.join(self.root, self.data_file.iloc[idx, 0])
    img = Image.open(img_path)
    label = self.data_file.iloc[idx, 1]

    if self.transform:
      img = self.transform(img)
  
    return img, label
  

In [ ]:
# creating columns
columns = ["Img_id", "Label"]
# training labels path
train_txt = "/content/labels/list_label_train.txt"
# reference : https://stackoverflow.com/questions/29287224/pandas-read-in-table-without-headers
train_set = pd.read_csv(train_txt, names = columns, header = None, delim_whitespace = True)
train_set["Img_id"] = train_set["Img_id"].apply(lambda i: i[:-4])
# adding "_aligned.jpg" files
train_set["Img_id"] = train_set["Img_id"] + "_aligned.jpg"
train_set["Label"] = train_set["Label"].apply(lambda i: i - 1)# reindexing

# texting labels path
test_txt = "/content/labels/list_label_test.txt"
# reference : https://stackoverflow.com/questions/29287224/pandas-read-in-table-without-headers
test_set = pd.read_csv(test_txt, names = columns, header = None, delim_whitespace = True)
test_set["Img_id"] = test_set["Img_id"].apply(lambda i: i[:-4])
# adding "_aligned.jpg" files
test_set["Img_id"] = test_set["Img_id"] + "_aligned.jpg"
test_set["Label"] = test_set["Label"].apply(lambda i: i - 1)# reindexing




In [ ]:
# (reference: Lab 08)
# We'll apply image changes to each loaded image.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_image_path = "/content/train" # train images path
test_image_path = "/content/test"   # test images path
train_img = CNN_func(data_file = train_set, root = train_image_path, transform = transform)
test_img = CNN_func(data_file = test_set,root = test_image_path, transform = transform)

In [ ]:
# (reference : Lab 08)
# dataloader helps us in handling our data
trainloader = torch.utils.data.DataLoader(train_img, batch_size=64,shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_img, batch_size=64,shuffle=False, num_workers=2)

In [ ]:
# (reference : Lab 08)
# define a Convolutional Neural Network
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 22 * 22 , 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 8)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), 16 * 22 *22)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# define cnn model
net = Net()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
count = 0
cnn_loss_list = []
iteration_list = []
cnn_accuracy_list = []
epochs = 30

for epoch in range(epochs):
    for i, (images, labels) in enumerate(trainloader):
            net = net.cuda()

            optimizer.zero_grad()
            outputs = net(images.cuda())
            cnn_loss = criterion(outputs.cuda(), labels.cuda())
            cnn_loss.backward()
            optimizer.step()

            count += 1

            if count % 50 == 0:
                correct = 0
                total = 0

                for i, (images, labels) in enumerate(testloader):

                    outputs = net(images.cuda())
                    predicted = torch.max(outputs.data, 1)[1]
                    total += len(labels)
                    correct += (predicted == labels.cuda()).sum()
                cnn_accuracy = 100 * correct / float(total)

                cnn_loss_list.append(cnn_loss.data)
                iteration_list.append(count)
                cnn_accuracy_list.append(cnn_accuracy)

                if count % 500 == 0:
                    print("Iteration: {} - Loss: {} - CNN Accuracy: {} %".format(count, cnn_loss.data, cnn_accuracy))

Iteration: 500 - Loss: 1.7525436878204346 - CNN Accuracy: 39.89569854736328 %
Iteration: 1000 - Loss: 1.0875189304351807 - CNN Accuracy: 57.65971374511719 %
Iteration: 1500 - Loss: 1.1163686513900757 - CNN Accuracy: 64.0482406616211 %
Iteration: 2000 - Loss: 0.8826941847801208 - CNN Accuracy: 64.92829132080078 %
Iteration: 2500 - Loss: 0.7190779447555542 - CNN Accuracy: 68.90482330322266 %
Iteration: 3000 - Loss: 0.49588510394096375 - CNN Accuracy: 67.50325775146484 %
Iteration: 3500 - Loss: 0.6414547562599182 - CNN Accuracy: 71.08865356445312 %
Iteration: 4000 - Loss: 0.44594240188598633 - CNN Accuracy: 71.41460418701172 %
Iteration: 4500 - Loss: 0.2987789511680603 - CNN Accuracy: 70.73011779785156 %
Iteration: 5000 - Loss: 0.23350264132022858 - CNN Accuracy: 71.80573272705078 %
Iteration: 5500 - Loss: 0.15876588225364685 - CNN Accuracy: 70.56714630126953 %


In [ ]:
from torchvision import models
alexnet_cnn = models.alexnet(pretrained = True) # define alexnet model
criterion = nn.CrossEntropyLoss().cuda()
optimizer_ft = optim.SGD(alexnet_cnn.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [ ]:
# (reference : https://ai-pool.com/a/s/end-to-end-pytorch-example-of-image-classification-with-convolutional-neural-networks)
count = 0
an_loss_list = []
iteration_list = []
an_acc_list = []
epochs = 30

for epoch in range(epochs):
    for i, (images, labels) in enumerate(trainloader):
            alexnet_cnn = alexnet_cnn.cuda()

            optimizer_ft.zero_grad()
            outputs = alexnet_cnn(images.cuda())
            an_loss = criterion(outputs.cuda(), labels.cuda())
            an_loss.backward()
            optimizer_ft.step()

            count += 1

            if count % 50 == 0:
                correct = 0
                total = 0

                for i, (images, labels) in enumerate(testloader):

                    outputs = alexnet_cnn(images.cuda())
                    predicted = torch.max(outputs.data, 1)[1]
                    total += len(labels)
                    correct += (predicted == labels.cuda()).sum()
                an_acc = 100 * correct / float(total)

                an_loss_list.append(an_loss.data)
                iteration_list.append(count)
                an_acc_list.append(an_acc)

                if count % 500 == 0:
                    print("Iteration: {} Loss: {} AlexNet Accuracy: {} %".format(count, an_loss.data, an_acc))

Iteration: 500 Loss: 1.0599859952926636 AlexNet Accuracy: 64.99347686767578 %
Iteration: 1000 Loss: 0.6581395864486694 AlexNet Accuracy: 69.06779479980469 %
Iteration: 1500 Loss: 0.5107983946800232 AlexNet Accuracy: 71.41460418701172 %
Iteration: 2000 Loss: 0.5949724912643433 AlexNet Accuracy: 71.44719696044922 %
Iteration: 2500 Loss: 0.2940003573894501 AlexNet Accuracy: 72.685791015625 %
Iteration: 3000 Loss: 0.31922370195388794 AlexNet Accuracy: 73.56584167480469 %
Iteration: 3500 Loss: 0.11006597429513931 AlexNet Accuracy: 74.73924255371094 %
Iteration: 4000 Loss: 0.133872851729393 AlexNet Accuracy: 73.40287017822266 %
Iteration: 4500 Loss: 0.06486912071704865 AlexNet Accuracy: 74.28292083740234 %
Iteration: 5000 Loss: 0.016297994181513786 AlexNet Accuracy: 74.28292083740234 %
Iteration: 5500 Loss: 0.015553765930235386 AlexNet Accuracy: 74.38069915771484 %


In [ ]:
vgg16_cnn = models.vgg16(pretrained = True) # define vgg16 model
criterion = nn.CrossEntropyLoss().cuda()
optimizer_ft = optim.SGD(vgg16_cnn.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
# ( reference : https://www.kaggle.com/code/carloalbertobarbano/vgg16-transfer-learning-pytorch/notebook)
count = 0
v_loss_ls = []
iteration_list = []
v_acc_list = []
epochs = 30

for epoch in range(epochs):
    for i, (images, labels) in enumerate(trainloader):
            vgg16_cnn = vgg16_cnn.cuda()

            optimizer_ft.zero_grad()
            outputs = vgg16_cnn(images.cuda())
            vgg_loss = criterion(outputs.cuda(), labels.cuda())
            vgg_loss.backward()
            optimizer_ft.step()

            count += 1

            if count % 50 == 0:
                correct = 0
                total = 0

                for i, (images, labels) in enumerate(testloader):

                    outputs = vgg16_cnn(images.cuda())
                    predicted = torch.max(outputs.data, 1)[1]
                    total += len(labels)
                    correct += (predicted == labels.cuda()).sum()
                v_acc = 100 * correct / float(total)

                v_loss_ls.append(vgg_loss.data)
                iteration_list.append(count)
                v_acc_list.append(v_acc)

                if count % 500 == 0:
                    print("Iteration: {} Loss: {} VGG16 accuracy: {} %".format(count, vgg_loss.data, v_acc))



Iteration: 500 Loss: 0.7769763469696045 VGG16 accuracy: 71.67535400390625 %
Iteration: 1000 Loss: 0.3454485833644867 VGG16 accuracy: 77.80313110351562 %
Iteration: 1500 Loss: 0.31379857659339905 VGG16 accuracy: 78.61798858642578 %
Iteration: 2000 Loss: 0.10812991112470627 VGG16 accuracy: 79.75879669189453 %
Iteration: 2500 Loss: 0.11473604291677475 VGG16 accuracy: 78.78096008300781 %
Iteration: 3000 Loss: 0.035595301538705826 VGG16 accuracy: 79.04171752929688 %
Iteration: 3500 Loss: 0.10835550725460052 VGG16 accuracy: 77.47718048095703 %
Iteration: 4000 Loss: 0.012578804045915604 VGG16 accuracy: 78.81356048583984 %
Iteration: 4500 Loss: 0.0006157935713417828 VGG16 accuracy: 80.70404052734375 %
Iteration: 5000 Loss: 0.008055316284298897 VGG16 accuracy: 80.67144775390625 %
Iteration: 5500 Loss: 0.00011378601629985496 VGG16 accuracy: 80.99739074707031 %


In [ ]:
# (reference : Lab 07)
from joblib import dump, load
dump(vgg16_cnn, 'CNN.joblib')

['CNN.joblib']